In [1]:
#imports
import datetime
import pandas as pd

#read google sheets 
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT6HrGh7EOzzejrvzkG_TGUM_GoGVDuvlUq7UcYqHlESZX6Vv8Hvwatsp4FLdE4Nmff9z5LSG3KQFq9/pub?gid=1362009325&single=true&output=csv')

#renamed columns
df.rename(columns = {'Rejection':'M/C', 'Unnamed: 6':'CASTING',
                              'Unnamed: 7':'OTHER','Timming':'Start_Time','Unnamed: 11':'End_Time','Total Prod':'Total_Prod','Final Prod':'Final_Prod','Total Rej':'Total_Rej'}, inplace = True)
df=df.drop(index=0)
df=df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'],format='%d-%m-%Y')  

# Convert the 'time_string' column to time object
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M')

# Calculate the time difference
 
df['time_difference1'] = df['End_Time'] - df['Start_Time'] 
df['time_difference1'] = df['time_difference1'] - pd.Timedelta(hours=1, minutes=30)
df['time_difference'] = df['time_difference1'].apply(lambda x: '{:02d}:{:02d}'.format(int(x.seconds // 3600), int((x.seconds // 60) % 60)))

df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M').dt.time
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M').dt.time
df['time_difference'] = pd.to_datetime(df['time_difference'], format='%H:%M').dt.time


job='S.R. Shaft THD'
breakTime='01:30'
start_date= '2023-06-01'
end_date= '2023-07-07'

# mask = (df['date'] > start_date) & (df['date'] <= end_date)
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
datedf=df.loc[mask]
newdf= datedf[datedf['Job']==job]
newdf=newdf.reset_index(drop=True)
total_time = newdf['time_difference1'].sum()
# df=df.drop(['time_difference1'], axis=1)
total_hours = total_time.total_seconds() / 3600
newdf = newdf.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
newdf['Date'] = newdf['Date'].dt.strftime('%d-%m-%Y') 
sumrow={'Date':'Total', 'Total_Prod':sum(newdf['Total_Prod']),'M/C':sum(newdf['M/C']),'CASTING':sum(newdf['CASTING']),'OTHER':sum(newdf['OTHER']),'Final_Prod':sum(newdf['Final_Prod']),'time_difference':total_hours}
sumrow= pd.DataFrame(sumrow,index=['Total'])
newdf = pd.concat([newdf, sumrow], axis=0)
machinedf=newdf.copy()
machinedf=machinedf.drop(['time_difference1'], axis=1)
machinedf = machinedf.fillna("...")

In [2]:
machinedf

,Date,Machines,Operator,Job,Total_Prod,M/C,CASTING,OTHER,Total_Rej,Final_Prod,Start_Time,End_Time,time_difference
0,02-06-2023,M/C No. 2,Sukh,S.R. Shaft THD,12,1,2,3,6.0,6,08:00:00,12:00:00,02:30:00
1,02-06-2023,M/C No. 2,Sukh,S.R. Shaft THD,15,6,0,1,7.0,8,08:30:00,13:30:00,03:30:00
2,03-07-2023,M/C No. 6,som,S.R. Shaft THD,9,0,1,0,1.0,8,08:10:00,21:30:00,11:50:00
3,05-07-2023,M/C No. 4,happy,S.R. Shaft THD,100,8,9,10,27.0,73,11:00:00,21:30:00,09:00:00
4,06-07-2023,M/C No. 5,happy,S.R. Shaft THD,100,8,9,10,27.0,73,11:50:00,22:30:00,09:10:00
5,07-07-2023,M/C No. 6,happy,S.R. Shaft THD,100,8,9,10,27.0,73,13:00:00,23:30:00,09:00:00
Total,Total,...,...,...,336,31,30,34,...,241,...,...,45.0


In [3]:
# Get the distinct values in a specific column
distinct_values = newdf['Operator'].unique()  # Replace 'column_name' with the actual column name

distinct_values=list(distinct_values)
distinct_values.pop()
days=[] 
total_hours =[]
sum_prod=[]
sum_mc=[]
sum_cast=[]
sum_other=[]
sum_finalprod=[]
avg_hourly=[]
avg_day=[]
for i in range(len(distinct_values)):
    op1=newdf[newdf['Operator']==distinct_values[i]]
    daysop=op1.shape[0]
    days.append(daysop)
    total_timeop = op1['time_difference1'].sum()
    total_hoursop = round((total_timeop.total_seconds() / 3600),2)
    total_hours.append(total_hoursop)
    sum_prodop=sum(op1['Total_Prod'])
    sum_prod.append(sum_prodop)
    sum_mcop=sum(op1['M/C'])
    sum_mc.append(sum_mcop)
    sum_castop=sum(op1['CASTING'])
    sum_cast.append(sum_castop)
    sum_otherop=sum(op1['OTHER'])
    sum_other.append(sum_otherop)
    sum_finalprodop=sum(op1['Final_Prod'])
    sum_finalprod.append(sum_finalprodop)
    avg_hourlyop=round((sum_prodop/total_hoursop),2)
    avg_hourly.append(avg_hourlyop)
    avg_dayop=round((sum_prodop/daysop),2)
    avg_day.append(avg_dayop)
product={
    'Operator':distinct_values,
    'DAYS':days,
    'HOURS': total_hours,
    'Total_Prod':sum_prod,
    'M/C':sum_mc,
    'Casting':sum_cast,
    'Other':sum_other,
    'Final_Prod':sum_finalprod,
    'Hourly_Avg':avg_hourly,
    'Day_Avg':avg_day
}
print(product)
prod_df=pd.DataFrame(product)
prod_df['Deduct_Machine_rej']=prod_df['Total_Prod']-prod_df['M/C']
prod_df['Deduct_Machine_rej']=prod_df['Deduct_Machine_rej'].round(2)
targetv=55
incentivev=0.70
prod_df['Hours X Target']=prod_df['HOURS']*targetv
prod_df['Hours X Target']=prod_df['Hours X Target'].round(2)
prod_df['Extra Prod']=prod_df['Deduct_Machine_rej']-prod_df['Hours X Target']
prod_df['Extra Prod']=prod_df['Extra Prod'].round(2)
prod_df['Incentive']=prod_df['Extra Prod']*incentivev
prod_df['Incentive']=prod_df['Incentive'].round(2)

sumrow2={'Operator':'Total', 'DAYS':sum(prod_df['DAYS']), 'HOURS':sum(prod_df['HOURS']), 'Total_Prod':sum(prod_df['Total_Prod']), 'M/C':sum(prod_df['M/C']),'Casting':sum(prod_df['Casting']),'Other':sum(prod_df['Other']),'Final_Prod':sum(prod_df['Final_Prod']), 'Hourly_Avg':round(sum(prod_df['Hourly_Avg']),2), 'Day_Avg':round(sum(prod_df['Day_Avg']),2), 'Deduct_Machine_rej':round(sum(prod_df['Deduct_Machine_rej']),2), 'Hours X Target':round(sum(prod_df['Hours X Target']),2), 'Extra Prod':round(sum(prod_df['Extra Prod']),2), 'Incentive':round(sum(prod_df['Incentive']),2)}
sumrow2= pd.DataFrame(sumrow2,index=['Total'])
operatordf = pd.concat([prod_df, sumrow2], axis=0)

{'Operator': ['Sukh', 'som', 'happy'], 'DAYS': [2, 1, 3], 'HOURS': [6.0, 11.83, 27.17], 'Total_Prod': [27, 9, 300], 'M/C': [7, 0, 24], 'Casting': [2, 1, 27], 'Other': [4, 0, 30], 'Final_Prod': [14, 8, 219], 'Hourly_Avg': [4.5, 0.76, 11.04], 'Day_Avg': [13.5, 9.0, 100.0]}


In [4]:
prod_df

,Operator,DAYS,HOURS,Total_Prod,M/C,Casting,Other,Final_Prod,Hourly_Avg,Day_Avg,Deduct_Machine_rej,Hours X Target,Extra Prod,Incentive
0,Sukh,2,6.00,27,7,2,4,14,4.50,13.5,20,330.00,-310.00,-217.00
1,som,1,11.83,9,0,1,0,8,0.76,9.0,9,650.65,-641.65,-449.16
2,happy,3,27.17,300,24,27,30,219,11.04,100.0,276,1494.35,-1218.35,-852.84


In [5]:
# Get the distinct values in a specific column
distinct_values_Job = df['Job'].unique()  
distinct_values_Job=list(distinct_values_Job)
distinct_values_Job.pop()

'ALU. BUSH FINAL'

In [6]:
#multi table
machinej=[]
t_prodj=[]
mcj=[]
castj=[]
otherj=[]
final=[]
column_names = ['Machines', 'PRODUCT', 'Total_Prod','M/C','CASTING','OTHER','FINAL']
dflist=[]
# Create an empty dataframe with the specified column names
jobs = pd.DataFrame(columns=column_names)

for j in range(len(distinct_values_Job)):
    job1=df[df['Job']==distinct_values_Job[j]]
    job1=job1.loc[mask]
    job1=job1.reset_index(drop=True)
    job1 = job1.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
    distinct_values_Mc = job1['Machines'].unique()
    distinct_values_Mc=list(distinct_values_Mc)
    
    jobs = pd.DataFrame(columns=column_names)
    
    for m in range(len(distinct_values_Mc)):
        job2=job1[job1['Machines']==distinct_values_Mc[m]]
        job2 = job2.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
        job3={'Machines':distinct_values_Mc[m], 'PRODUCT':distinct_values_Job[j], 'Total_Prod': sum(job2['Total_Prod']), 'M/C':sum(job2['M/C']), 'CASTING':sum(job2['CASTING']), 'OTHER': sum(job2['OTHER']), 'FINAL':sum(job2['Final_Prod'])}
        job3=pd.DataFrame(job3,index=[m])
        jobs=pd.concat([jobs,job3],ignore_index=True)
    job3={'Machines':'Total', 'Total_Prod':sum(jobs['Total_Prod']), 'M/C':sum(jobs['M/C']), 'CASTING':sum(jobs['CASTING']), 'OTHER':sum(jobs['OTHER']), 'FINAL':sum(jobs['FINAL'])}
    job3=pd.DataFrame(job3,index=[0])
    jobs=pd.concat([jobs,job3],ignore_index=True)
    dflist.append(jobs)

In [7]:
dflist

[    Machines PRODUCT Total_Prod M/C CASTING OTHER FINAL
 0  M/C No. 2    test         35  11       7     6    11
 1      Total     NaN         35  11       7     6    11,
     Machines          PRODUCT Total_Prod M/C CASTING OTHER FINAL
 0  M/C No. 7  ALU. BUSH ROUGH         10   1       2     3     4
 1      Total              NaN         10   1       2     3     4,
     Machines         PRODUCT Total_Prod M/C CASTING OTHER FINAL
 0  M/C No. 2  S.R. Shaft THD         27   7       2     4    14
 1  M/C No. 6  S.R. Shaft THD        109   8      10    10    81
 2  M/C No. 4  S.R. Shaft THD        100   8       9    10    73
 3  M/C No. 5  S.R. Shaft THD        100   8       9    10    73
 4      Total             NaN        336  31      30    34   241]

## Operator page logic####

In [8]:
#imports
import datetime
import pandas as pd

#read google sheets 
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT6HrGh7EOzzejrvzkG_TGUM_GoGVDuvlUq7UcYqHlESZX6Vv8Hvwatsp4FLdE4Nmff9z5LSG3KQFq9/pub?gid=1362009325&single=true&output=csv')

#renamed columns
df.rename(columns = {'Rejection':'M/C', 'Unnamed: 6':'CASTING',
                              'Unnamed: 7':'OTHER','Timming':'Start_Time','Unnamed: 11':'End_Time','Total Prod':'Total_Prod','Final Prod':'Final_Prod','Total Rej':'Total_Rej'}, inplace = True)
df=df.drop(index=0)
df=df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'],format='%d-%m-%Y')  

# Convert the 'time_string' column to time object
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M')

# Calculate the time difference
 
df['time_difference1'] = df['End_Time'] - df['Start_Time'] 
df['time_difference1'] = df['time_difference1'] - pd.Timedelta(hours=1, minutes=30)
df['time_difference'] = df['time_difference1'].apply(lambda x: '{:02d}:{:02d}'.format(int(x.seconds // 3600), int((x.seconds // 60) % 60)))

df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M').dt.time
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M').dt.time
df['time_difference'] = pd.to_datetime(df['time_difference'], format='%H:%M').dt.time


job='S.R. Shaft THD'
breakTime='01:30'
start_date= '2023-06-01'
end_date= '2023-07-07'

# mask = (df['date'] > start_date) & (df['date'] <= end_date)
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
datedf=df.loc[mask]
newdf= datedf[datedf['Job']==job]
newdf=newdf.reset_index(drop=True)
total_time = newdf['time_difference1'].sum()
# df=df.drop(['time_difference1'], axis=1)
total_hours = total_time.total_seconds() / 3600
newdf = newdf.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
newdf['Date'] = newdf['Date'].dt.strftime('%d-%m-%Y') 
sumrow={'Date':'Total', 'Total_Prod':sum(newdf['Total_Prod']),'M/C':sum(newdf['M/C']),'CASTING':sum(newdf['CASTING']),'OTHER':sum(newdf['OTHER']),'Final_Prod':sum(newdf['Final_Prod']),'time_difference':total_hours}
sumrow= pd.DataFrame(sumrow,index=['Total'])
newdf = pd.concat([newdf, sumrow], axis=0)

# Get the distinct values in a specific column
distinct_values = newdf['Operator'].unique()  # Replace 'column_name' with the actual column name

distinct_values=list(distinct_values)
distinct_values.pop()
days=[] 
total_hours =[]
sum_prod=[]
sum_mc=[]
sum_cast=[]
sum_other=[]
sum_finalprod=[]
avg_hourly=[]
avg_day=[]
for i in range(len(distinct_values)):
    op1=newdf[newdf['Operator']==distinct_values[i]]
    daysop=op1.shape[0]
    days.append(daysop)
    total_timeop = op1['time_difference1'].sum()
    total_hoursop = round((total_timeop.total_seconds() / 3600),2)
    total_hours.append(total_hoursop)
    sum_prodop=sum(op1['Total_Prod'])
    sum_prod.append(sum_prodop)
    sum_mcop=sum(op1['M/C'])
    sum_mc.append(sum_mcop)
    sum_castop=sum(op1['CASTING'])
    sum_cast.append(sum_castop)
    sum_otherop=sum(op1['OTHER'])
    sum_other.append(sum_otherop)
    sum_finalprodop=sum(op1['Final_Prod'])
    sum_finalprod.append(sum_finalprodop)
    avg_hourlyop=round((sum_prodop/total_hoursop),2)
    avg_hourly.append(avg_hourlyop)
    avg_dayop=round((sum_prodop/daysop),2)
    avg_day.append(avg_dayop)
product={
    'Operator':distinct_values,
    'DAYS':days,
    'HOURS': total_hours,
    'Total_Prod':sum_prod,
    'M/C':sum_mc,
    'Casting':sum_cast,
    'Other':sum_other,
    'Final_Prod':sum_finalprod,
    'Hourly_Avg':avg_hourly,
    'Day_Avg':avg_day
}
print(product)
prod_df=pd.DataFrame(product)
prod_df['Deduct_Machine_rej']=prod_df['Total_Prod']-prod_df['M/C']
prod_df['Deduct_Machine_rej']=prod_df['Deduct_Machine_rej'].round(2)
targetv=55
incentivev=0.70
prod_df['Hours X Target']=prod_df['HOURS']*targetv
prod_df['Hours X Target']=prod_df['Hours X Target'].round(2)
prod_df['Extra Prod']=prod_df['Deduct_Machine_rej']-prod_df['Hours X Target']
prod_df['Extra Prod']=prod_df['Extra Prod'].round(2)
prod_df['Incentive']=prod_df['Extra Prod']*incentivev
prod_df['Incentive']=prod_df['Incentive'].round(2)

sumrow2={'Operator':'Total', 'DAYS':sum(prod_df['DAYS']), 'HOURS':sum(prod_df['HOURS']), 'Total_Prod':sum(prod_df['Total_Prod']), 'M/C':sum(prod_df['M/C']),'Casting':sum(prod_df['Casting']),'Other':sum(prod_df['Other']),'Final_Prod':sum(prod_df['Final_Prod']), 'Hourly_Avg':round(sum(prod_df['Hourly_Avg']),2), 'Day_Avg':round(sum(prod_df['Day_Avg']),2), 'Deduct_Machine_rej':round(sum(prod_df['Deduct_Machine_rej']),2), 'Hours X Target':round(sum(prod_df['Hours X Target']),2), 'Extra Prod':round(sum(prod_df['Extra Prod']),2), 'Incentive':round(sum(prod_df['Incentive']),2)}
sumrow2= pd.DataFrame(sumrow2,index=['Total'])
operatordf = pd.concat([prod_df, sumrow2], axis=0)

{'Operator': ['Sukh', 'som', 'happy'], 'DAYS': [2, 1, 3], 'HOURS': [6.0, 11.83, 27.17], 'Total_Prod': [27, 9, 300], 'M/C': [7, 0, 24], 'Casting': [2, 1, 27], 'Other': [4, 0, 30], 'Final_Prod': [14, 8, 219], 'Hourly_Avg': [4.5, 0.76, 11.04], 'Day_Avg': [13.5, 9.0, 100.0]}


In [9]:
operatordf

,Operator,DAYS,HOURS,Total_Prod,M/C,Casting,Other,Final_Prod,Hourly_Avg,Day_Avg,Deduct_Machine_rej,Hours X Target,Extra Prod,Incentive
0,Sukh,2,6.00,27,7,2,4,14,4.50,13.5,20,330.00,-310.00,-217.00
1,som,1,11.83,9,0,1,0,8,0.76,9.0,9,650.65,-641.65,-449.16
2,happy,3,27.17,300,24,27,30,219,11.04,100.0,276,1494.35,-1218.35,-852.84
Total,Total,6,45.00,336,31,30,34,241,16.30,122.5,305,2475.00,-2170.00,-1519.00


### Monthly Data

In [ ]:
#imports
import datetime
import pandas as pd

#read google sheets 
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT6HrGh7EOzzejrvzkG_TGUM_GoGVDuvlUq7UcYqHlESZX6Vv8Hvwatsp4FLdE4Nmff9z5LSG3KQFq9/pub?gid=1362009325&single=true&output=csv')

#renamed columns
df.rename(columns = {'Rejection':'M/C', 'Unnamed: 6':'CASTING',
                              'Unnamed: 7':'OTHER','Timming':'Start_Time','Unnamed: 11':'End_Time','Total Prod':'Total_Prod','Final Prod':'Final_Prod','Total Rej':'Total_Rej'}, inplace = True)
df=df.drop(index=0)
df=df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'],format='%d-%m-%Y')  

# Convert the 'time_string' column to time object
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M')
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M')

# Calculate the time difference
 
df['time_difference1'] = df['End_Time'] - df['Start_Time'] 
df['time_difference1'] = df['time_difference1'] - pd.Timedelta(hours=1, minutes=30)
df['time_difference'] = df['time_difference1'].apply(lambda x: '{:02d}:{:02d}'.format(int(x.seconds // 3600), int((x.seconds // 60) % 60)))

df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='%H:%M').dt.time
df['End_Time'] = pd.to_datetime(df['End_Time'], format='%H:%M').dt.time
df['time_difference'] = pd.to_datetime(df['time_difference'], format='%H:%M').dt.time


job='S.R. Shaft THD'
breakTime='01:30'
start_date= '2023-06-01'
end_date= '2023-07-07'

# mask = (df['date'] > start_date) & (df['date'] <= end_date)
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
datedf=df.loc[mask]
newdf= datedf[datedf['Job']==job]
newdf=newdf.reset_index(drop=True)
total_time = newdf['time_difference1'].sum()
# df=df.drop(['time_difference1'], axis=1)
total_hours = total_time.total_seconds() / 3600
newdf = newdf.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
newdf['Date'] = newdf['Date'].dt.strftime('%d-%m-%Y') 
sumrow={'Date':'Total', 'Total_Prod':sum(newdf['Total_Prod']),'M/C':sum(newdf['M/C']),'CASTING':sum(newdf['CASTING']),'OTHER':sum(newdf['OTHER']),'Final_Prod':sum(newdf['Final_Prod']),'time_difference':total_hours}
sumrow= pd.DataFrame(sumrow,index=['Total'])
newdf = pd.concat([newdf, sumrow], axis=0)

In [ ]:
# Get the distinct values in a specific column
distinct_values_Job = newdf['Job'].unique()  
distinct_values_Job=list(distinct_values_Job)
distinct_values_Job.pop()

In [ ]:
#multi table
machinej=[]
t_prodj=[]
mcj=[]
castj=[]
otherj=[]
final=[]
column_names = ['Machines', 'PRODUCT', 'Total_Prod','M/C','CASTING','OTHER','FINAL']
dflist=[]
# Create an empty dataframe with the specified column names
jobs = pd.DataFrame(columns=column_names)

for j in range(len(distinct_values_Job)):
    job1=newdf[newdf['Job']==distinct_values_Job[j]]
    job1=job1.loc[mask]
    job1=job1.reset_index(drop=True)
    job1 = job1.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
    distinct_values_Mc = job1['Machines'].unique()
    distinct_values_Mc=list(distinct_values_Mc)
    
    jobs = pd.DataFrame(columns=column_names)
    
    for m in range(len(distinct_values_Mc)):
        job2=job1[job1['Machines']==distinct_values_Mc[m]]
        job2 = job2.astype({'Total_Prod':'int','M/C':'int','CASTING':'int','OTHER':'int','Total_Rej':'int','Final_Prod':'int'})
        job3={'Machines':distinct_values_Mc[m], 'PRODUCT':distinct_values_Job[j], 'Total_Prod': sum(job2['Total_Prod']), 'M/C':sum(job2['M/C']), 'CASTING':sum(job2['CASTING']), 'OTHER': sum(job2['OTHER']), 'FINAL':sum(job2['Final_Prod'])}
        job3=pd.DataFrame(job3,index=[m])
        jobs=pd.concat([jobs,job3],ignore_index=True)
    job3={'Machines':'Total', 'Total_Prod':sum(jobs['Total_Prod']), 'M/C':sum(jobs['M/C']), 'CASTING':sum(jobs['CASTING']), 'OTHER':sum(jobs['OTHER']), 'FINAL':sum(jobs['FINAL'])}
    job3=pd.DataFrame(job3,index=[0])
    jobs=pd.concat([jobs,job3],ignore_index=True)
    dflist.append(jobs)